# Pendahuluan Karakter Optik

![Robot membaca koran](./images/ocr.jpg)

Tantangan visi komputer yang umum adalah mendeteksi dan menafsirkan teks pada gambar. Pemrosesan seperti ini ini sering disebut sebagai *Pendahuluan Karakter Optik (OCR)*.

## Menggunakan Layanan Visi Komputer untuk Membaca Teks dalam Gambar

Layanan kognitif **Visi Komputer** menyediakan dukungan untuk tugas-tugas OCR, termasuk:

- API **OCR** dapat Anda gunakan untuk membaca teks dalam beberapa bahasa. API ini dapat digunakan secara sinkron, dan berfungsi dengan baik saat Anda perlu mendeteksi dan membaca sejumlah kecil teks dalam sebuah gambar.
- API **Baca** yang dioptimalkan untuk dokumen yang lebih besar. API ini digunakan secara asinkron, dan dapat digunakan untuk teks cetak dan tulisan tangan.

Anda dapat menggunakan layanan ini dengan membuat sumber daya **Visi Komputer** atau sumber daya **Cognitive Services**.

Jika Anda belum melakukannya, buat sumber daya **Cognitive Services** di langganan Azure Anda.

> **Catatan**: Jika Anda sudah memiliki sumber daya Cognitive Services, cukup buka halaman Mulai cepat di portal Microsoft Azure dan salin kunci dan titik akhirnya ke sel di bawah. Atau, ikuti langkah-langkah di bawah untuk membuatnya.

1. Di tab browser lain, buka portal Microsoft Azure  di https://portal.azure.com, masuk menggunakan akun Microsoft Anda.

2. Klik tombol **&#65291;Buat sumber daya**, cari *Cognitive Services*, dan buat sumber daya **Cognitive Services** dengan pengaturan berikut:
    - **Langganan**: *Langganan Azure Anda*.
    - **Grup sumber daya**: *Pilih atau buat grup sumber daya dengan nama unik*.
    - **Wilayah**: *Pilih wilayah yang tersedia*:
    - **Nama**: *Masukkan nama yang unik*.
    - **Tingkat Harga**: S0
    - **Saya mengonfirmasi bahwa saya telah membaca dan memahami pemberitahuan tersebut**: Dipilih.
3. Tunggu penyebaran hingga selesai. Lalu, buka sumber daya layanan kognitif, dan di halaman **Ringkasan**, klik tautan untuk mengelola kunci layanan. Anda akan memerlukan titik akhir dan kunci untuk terhubung ke sumber daya layanan kognitif Anda dari aplikasi klien.

### Mendapatkan Kunci dan Titik Akhir untuk sumber daya Cognitive Services Anda

Untuk menggunakan sumber daya layanan kognitif, aplikasi klien memerlukan titik akhir dan kunci autentikasi:

1. Di portal Azure, di halaman **Kunci dan Titik Akhir** untuk sumber daya layanan kognitif Anda, salin **Kunci1** untuk sumber daya dan tempel pada kode di bawah, menggantikan **YOUR_COG_KEY**.
2. Salin **titik akhir** untuk sumber daya dan tempel pada kode di bawah, menggantikan **YOUR_COG_ENDPOINT**.
3. Jalankan kode pada sel di bawah dengan mengeklik tombol **Jalankan sel** (&#9655;) (ke sebelah kiri sel).

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Setelah menyiapkan kunci dan titik akhir, Anda dapat menggunakan sumber daya layanan visi komputer untuk mengekstrak teks gambar.

Mari kita mulai dengan API **OCR**, yang memungkinkan Anda menganalisis gambar secara sinkron dan membaca isi teksnya. Dalam hal ini, Anda memiliki gambar iklan untuk perusahaan ritel Northwind Traders fiktif yang menyertakan beberapa teks. Jalankan sel di bawah untuk membacanya. 

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Read the image file
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

# Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

Teks yang ditemukan dalam gambar diatur ke dalam struktur hierarki wilayah, garis, dan kata, dan kode membacanya untuk mengambil hasilnya.

Di hasil, lihat teks yang dibaca di atas gambar. 

## Menampilkan kotak pembatas

Hasil tersebut juga mencakup koordinat *kotak pembatas* untuk baris teks dan masing-masing kata yang ditemukan dalam gambar. Jalankan sel di bawah ini untuk melihat kotak pembatas untuk baris teks pada gambar iklan yang Anda ambil di atas.

In [ ]:
# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Show the position of the line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Show the image with the text locations highlighted
plt.axis('off')
plt.imshow(img)

Hasilnya, kotak pembatas untuk setiap baris teks ditampilkan sebagai persegi panjang pada gambar.

## Menggunakan API Baca

API OCR yang Anda gunakan sebelumnya berfungsi dengan baik untuk gambar dengan sedikit teks. Saat Anda harus membaca lebih banyak teks, seperti dokumen yang dipindai, Anda dapat menggunakan API **Baca**. Hal ini memerlukan proses beberapa langkah:

1. Kirim gambar ke layanan Visi Komputer untuk dibaca dan dianalisis secara asinkron.
2. Tunggu hingga operasi analisis selesai.
3. Ambil hasil analisis tersebut.

Jalankan sel berikut untuk menggunakan proses ini untuk membaca teks dalam surat yang dipindai ke pengelola toko Northwind Traders.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

Tinjau hasilnya. Ada transkripsi lengkap surat tersebut, yang sebagian besar berisi teks cetak dengan tanda tangan tulisan tangan. Gambar asli surat ditampilkan di bawah hasil OCR (Anda harus menggulir ke bawah untuk melihatnya).

## Membaca teks tulisan tangan

Pada contoh sebelumnya, permintaan untuk menganalisis gambar menentukan mode pengenalan teks yang mengoptimalkan operasi untuk teks yang *dicetak*. Perhatikan bahwa meskipun demikian, tanda tangan tulisan tangan ini bisa terbaca.

Kemampuan untuk membaca teks tulisan tangan ini sangat bermanfaat. Misalnya, Anda menulis catatan yang berisi daftar belanja, dan ingin menggunakan aplikasi di ponsel untuk membaca catatan tersebut dan mentranskripsikan teks di dalamnya.

Jalankan sel di bawah untuk melihat contoh operasi baca untuk daftar belanja tulisan tangan.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Informasi Selengkapnya

Untuk informasi selengkapnya tentang penggunaan layanan Visi Komputer untuk OCR, lihat [dokumentasi Visi Komputer](https://docs.microsoft.com/id-id/azure/cognitive-services/computer-vision/concept-recognizing-text)